<a href="https://colab.research.google.com/github/ary3120-droid/myproject/blob/main/MMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

file_path = '/content/drive/MyDrive/mercari-price-suggestion-challenge/train.tsv/train.tsv'

mercari_df = pd.read_csv(file_path, sep='\t')

name 컬럼 정제: 특수문자와 과도한 공백을 제거하여 name_clean 컬럼을 생성

In [6]:
import re
import pandas as pd

def clean_name_remove_size(text):
    if pd.isna(text):
        return ""

    text = str(text)

    # 특수문자 → 공백 (알파벳/숫자만 남기기)
    text = re.sub(r"[^a-zA-Z0-9]+", " ", text)

    # 공백 정리
    text = re.sub(r"\s+", " ", text).strip()

    return text


def apply_clean_name(mercari_df):
    mercari_df["name_clean"] = mercari_df["name"].fillna("").apply(clean_name_remove_size)
    return mercari_df


결측 브랜드 채우기

In [7]:
def build_token_brand_map(mercari_df, min_count=20, min_ratio=0.8):
    """
    mercari_df: 전처리된 train 데이터프레임 (name_clean 포함)
    min_count: 해당 토큰으로 등장한 브랜드 수 최소 빈도
    min_ratio: 해당 토큰에서 특정 브랜드가 차지하는 비율 (precision 기준)

    반환:
      tok1_map, tok2_map : {
          토큰 : 대표 브랜드
      } 형태의 dict
    """

    # 1) brand_name 이 존재하는 경우만 학습에 사용
    df_known = mercari_df[~mercari_df["brand_name"].isna()].copy()

    # 2) name_clean 기준으로 토큰 분리
    tokens = df_known["name_clean"].str.split()
    df_known["tok1"] = tokens.str[0]                 # 첫 단어
    df_known["tok2"] = tokens.str[:2].str.join(" ")  # 첫 두 단어

    # 3) tok1/tok2 각각 매핑 생성
    tok_maps = {}

    for col in ["tok1", "tok2"]:
        # (토큰, 브랜드) 조합별 등장 횟수
        grp = (
            df_known
            .groupby([col, "brand_name"])
            .size()
            .reset_index(name="cnt")
        )

        # 해당 토큰의 전체 등장 횟수
        total = grp.groupby(col)["cnt"].sum().reset_index(name="total")
        grp = grp.merge(total, on=col, how="left")

        # ratio = 특정 브랜드가 tok 전체에서 차지하는 비율
        grp["ratio"] = grp["cnt"] / grp["total"]

        # 필터링: 자주 나오고 압도적 비율일 때만 인정
        cond = (grp["cnt"] >= min_count) & (grp["ratio"] >= min_ratio)
        grp_strong = grp[cond].copy()

        # 동일 토큰에 여러 브랜드가 남아 있으면 → 가장 ratio 높은 브랜드만 선택
        grp_strong = (
            grp_strong
            .sort_values(["ratio", "cnt"], ascending=[False, False])
            .drop_duplicates(subset=[col], keep="first")
        )

        # dict 매핑
        tok_maps[col] = dict(zip(grp_strong[col], grp_strong["brand_name"]))

    # 4) 최종 dict 분리
    tok1_map = tok_maps["tok1"]
    tok2_map = tok_maps["tok2"]

    return tok1_map, tok2_map

In [8]:
def fill_brand_by_token_map(mercari_df, tok1_map, tok2_map):
    """
    mercari_df : 브랜드를 채울 대상 데이터프레임 (name_clean, brand_name 포함)
    tok1_map   : 첫 단어(tok1) → 브랜드 매핑 dict
    tok2_map   : 첫 두 단어(tok2) → 브랜드 매핑 dict

    반환:
      brand_filled 컬럼이 추가된 mercari_df 복사본
        - 원래 brand_name 있으면 그대로 유지
        - 결측일 때만 tok2 → tok1 순서로 채움
        - 그래도 없으면 'unknown'
    """
    df = mercari_df.copy()

    # name_clean 기준 토큰 분리
    tokens = df["name_clean"].str.split()
    df["tok1"] = tokens.str[0]                   # 첫 단어
    df["tok2"] = tokens.str[:2].str.join(" ")    # 첫 두 단어

    # 새 컬럼: 기본값은 원래 brand_name (문자열화)
    df["brand_filled"] = df["brand_name"].astype(str)

    # 채울 대상: brand_name 이 NaN 이거나 완전 공백
    mask_missing = df["brand_name"].isna() | (df["brand_name"].astype(str).str.strip() == "")

    # 1순위: tok2 매핑이 있는 경우 (더 정확하니까 먼저)
    mask_tok2 = mask_missing & df["tok2"].isin(tok2_map.keys())
    df.loc[mask_tok2, "brand_filled"] = df.loc[mask_tok2, "tok2"].map(tok2_map)

    # 2순위: 여전히 비어 있는 경우 tok1 매핑 사용
    mask_missing2 = (
        df["brand_filled"].isna()
        | (df["brand_filled"].astype(str).str.strip() == "")
        | (df["brand_filled"] == "nan")
    )
    mask_tok1 = mask_missing2 & df["tok1"].isin(tok1_map.keys())
    df.loc[mask_tok1, "brand_filled"] = df.loc[mask_tok1, "tok1"].map(tok1_map)

    # 그래도 비어 있으면 unknown 으로 통일
    df["brand_filled"] = df["brand_filled"].replace(["", "nan"], "unknown")
    df["brand_filled"] = df["brand_filled"].fillna("unknown")

    # 임시 컬럼 정리
    df = df.drop(columns=["tok1", "tok2"])
    return df



In [9]:

# 1) name_clean 만들기
mercari_df = apply_clean_name(mercari_df)

# 2) 토큰 → 브랜드 매핑 생성 (train 기준)
tok1_map, tok2_map = build_token_brand_map(mercari_df, min_count=20, min_ratio=0.8)

# 3) 브랜드 결측 채우기
mercari_df = fill_brand_by_token_map(mercari_df, tok1_map, tok2_map)

# 4) 결과 확인
mercari_df[["name", "name_clean", "brand_name", "brand_filled"]].head(20)


,name,name_clean,brand_name,brand_filled
0,MLB Cincinnati Reds T Shirt Size XL,MLB Cincinnati Reds T Shirt Size XL,NaN,unknown
1,Razer BlackWidow Chroma Keyboard,Razer BlackWidow Chroma Keyboard,Razer,Razer
2,AVA-VIV Blouse,AVA VIV Blouse,Target,Target
3,Leather Horse Statues,Leather Horse Statues,NaN,unknown
4,24K GOLD plated rose,24K GOLD plated rose,NaN,unknown
5,Bundled items requested for Ruie,Bundled items requested for Ruie,NaN,unknown
6,Acacia pacific tides santorini top,Acacia pacific tides santorini top,Acacia Swimwear,Acacia Swimwear
7,Girls cheer and tumbling bundle of 7,Girls cheer and tumbling bundle of 7,Soffe,Soffe
8,Girls Nike Pro shorts,Girls Nike Pro shorts,Nike,Nike
9,Porcelain clown doll checker pants VTG,Porcelain clown doll checker pants VTG,NaN,unknown


In [ ]:
mercari_df["brand_name"] = mercari_df["brand_name"].str.lower()

브랜드 자동 채우기 정확도를 높이기 위해 이름(name_clean)에서 자주 등장하는 토큰과 브랜드의 관계를 통계적으로 분석

In [10]:
def build_token_brand_map_with_stats(mercari_df, min_count=20, min_ratio=0.8):
    """
    mercari_df: 전처리된 train 데이터프레임 (name_clean, brand_name 포함)
    min_count : (토큰, 브랜드) 조합의 최소 등장 횟수
    min_ratio : 특정 브랜드가 해당 토큰에서 차지하는 비율 (precision 기준)

    반환:
      tok1_map : 첫 단어(tok1)  -> 대표 브랜드 dict
      tok2_map : 첫 두 단어(tok2) -> 대표 브랜드 dict
      tok1_df  : tok1 기준 (토큰, 브랜드, cnt, total, ratio) 테이블
      tok2_df  : tok2 기준 (토큰, 브랜드, cnt, total, ratio) 테이블
    """

    # name_clean 없는 상태에서 호출하면 에러를 명확하게 내주기
    if "name_clean" not in mercari_df.columns:
        raise ValueError("name_clean 컬럼이 없습니다. 먼저 apply_clean_name()을 실행해 주세요.")

    # 1) brand_name 이 있는 행만 사용
    df_known = mercari_df[~mercari_df["brand_name"].isna()].copy()

    # 2) tok1, tok2 생성
    tokens = df_known["name_clean"].str.split()
    df_known["tok1"] = tokens.str[0]                 # 첫 단어
    df_known["tok2"] = tokens.str[:2].str.join(" ")  # 첫 두 단어

    tok_maps = {}
    tok_stats = {}

    # 3) tok1 / tok2 에 대해 동일 로직 수행
    for col in ["tok1", "tok2"]:
        # (토큰, 브랜드)별 카운트
        grp = (
            df_known
            .groupby([col, "brand_name"])
            .size()
            .reset_index(name="cnt")
        )

        # 토큰별 전체 카운트
        total = grp.groupby(col)["cnt"].sum().reset_index(name="total")
        grp = grp.merge(total, on=col, how="left")

        # 비율 계산: 해당 토큰에서 이 브랜드가 차지하는 비율
        grp["ratio"] = grp["cnt"] / grp["total"]

        # 조건: 자주 등장(cnt) + 한 브랜드가 압도적(ratio)
        cond = (grp["cnt"] >= min_count) & (grp["ratio"] >= min_ratio)
        grp_strong = grp[cond].copy()

        # 같은 토큰에 여러 브랜드가 남아 있으면
        # ratio, cnt 기준으로 가장 강한 것만 남김
        grp_strong = (
            grp_strong
            .sort_values(["ratio", "cnt"], ascending=[False, False])
            .drop_duplicates(subset=[col], keep="first")
        )

        # dict 매핑과 통계 DF 저장
        tok_maps[col] = dict(zip(grp_strong[col], grp_strong["brand_name"]))
        tok_stats[col] = grp_strong

    tok1_map = tok_maps["tok1"]
    tok2_map = tok_maps["tok2"]
    tok1_df  = tok_stats["tok1"]   # 컬럼: tok1, brand_name, cnt, total, ratio
    tok2_df  = tok_stats["tok2"]   # 컬럼: tok2, brand_name, cnt, total, ratio

    return tok1_map, tok2_map, tok1_df, tok2_df


In [11]:
tok1_map, tok2_map, tok1_df, tok2_df = build_token_brand_map_with_stats(
    mercari_df,
    min_count=20,
    min_ratio=0.8
)


In [12]:
print("원본 브랜드 결측률:", mercari_df["brand_name"].isna().mean())
print("보정 후 unknown 비율:", (mercari_df["brand_filled"]=="unknown").mean())
print("자동 보정된 브랜드 수:", ((mercari_df["brand_name"].isna()) & (mercari_df["brand_filled"]!="unknown")).sum())


원본 브랜드 결측률: 0.4267568725190299
보정 후 unknown 비율: 0.3622882427733578
자동 보정된 브랜드 수: 95577


In [13]:
mercari_df.head(20)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,name_clean,brand_filled
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet,MLB Cincinnati Reds T Shirt Size XL,unknown
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,Razer BlackWidow Chroma Keyboard,Razer
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,AVA VIV Blouse,Target
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,Leather Horse Statues,unknown
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity,24K GOLD plated rose,unknown
5,5,Bundled items requested for Ruie,3,Women/Other/Other,NaN,59.0,0,"Banana republic bottoms, Candies skirt with ma...",Bundled items requested for Ruie,unknown
6,6,Acacia pacific tides santorini top,3,Women/Swimwear/Two-Piece,Acacia Swimwear,64.0,0,Size small but straps slightly shortened to fi...,Acacia pacific tides santorini top,Acacia Swimwear
7,7,Girls cheer and tumbling bundle of 7,3,Sports & Outdoors/Apparel/Girls,Soffe,6.0,1,You get three pairs of Sophie cheer shorts siz...,Girls cheer and tumbling bundle of 7,Soffe
8,8,Girls Nike Pro shorts,3,Sports & Outdoors/Apparel/Girls,Nike,19.0,0,Girls Size small Plus green. Three shorts total.,Girls Nike Pro shorts,Nike
9,9,Porcelain clown doll checker pants VTG,3,Vintage & Collectibles/Collectibles/Doll,NaN,8.0,0,I realized his pants are on backwards after th...,Porcelain clown doll checker pants VTG,unknown


In [14]:
mercari_df.columns

Index(['train_id', 'name', 'item_condition_id', 'category_name', 'brand_name',
       'price', 'shipping', 'item_description', 'name_clean', 'brand_filled'],
      dtype='object')

ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

In [15]:
import re
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

file_path = '/content/drive/MyDrive/mercari-price-suggestion-challenge/train.tsv/train.tsv'
mercari_df = pd.read_csv(file_path, sep='\t')

카테고리 split + 라벨인코딩

In [17]:
def split_category(df):
    df["category_name"] = df["category_name"].fillna("unknown")

    def _split(x):
        parts = str(x).split("/")
        parts = parts + ["unknown"] * (3 - len(parts))   # 부족하면 unknown 채우기
        return parts[:3]  # 앞에서 3개만

    df[["cat1", "cat2", "cat3"]] = df["category_name"].apply(lambda x: pd.Series(_split(x)))
    return df


In [18]:
from sklearn.preprocessing import LabelEncoder

def label_encode_columns(df, cols):
    for col in cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
    return df


텍스트 기반 피처 만들기 (Ridge OOF)

In [29]:
import re
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def build_features(df):
    df = df.copy()

    # -------------------------
    # 1) 기본 결측 처리
    # -------------------------
    df["name"] = df["name"].fillna("")
    df["item_description"] = df["item_description"].fillna("")
    df["brand_name"] = df["brand_name"].fillna("unknown").str.lower()
    df["category_name"] = df["category_name"].fillna("unknown")

    # -------------------------
    # 2) 카테고리 split
    # -------------------------
    def _split(x):
        parts = str(x).split("/")
        parts = parts + ["unknown"] * (3 - len(parts))
        return parts[:3]

    df[["cat1", "cat2", "cat3"]] = df["category_name"].apply(lambda x: pd.Series(_split(x)))

    # -------------------------
    # 3) 브랜드 보정
    # -------------------------
    df["brand_filled"] = df["brand_name"].replace("", "unknown")

    # -------------------------
    # 4) 이름 기반 피처
    # -------------------------
    df["name_len"] = df["name"].str.len()
    df["name_word_count"] = df["name"].str.split().apply(len)

    # 특수문자 / 숫자
    df["name_special_count"] = df["name"].str.count(r"[^0-9A-Za-z\s]")
    df["name_special"] = (df["name_special_count"] > 0).astype(int)

    df["name_digit_count"] = df["name"].str.count(r"\d")
    df["name_digit"] = (df["name_digit_count"] > 0).astype(int)

    # rm 패턴
    df["contains_rm"] = df["name"].str.lower().str.contains(r"rm|[$]").astype(int)

    # -------------------------
    # 5) 설명 기반 피처
    # -------------------------
    df["desc_len"] = df["item_description"].str.len()
    df["desc_word_count"] = df["item_description"].str.split().apply(len)

    df["desc_special_count"] = df["item_description"].str.count(r"[^0-9A-Za-z\s]")
    df["desc_special"] = (df["desc_special_count"] > 0).astype(int)

    df["desc_digit_count"] = df["item_description"].str.count(r"\d")
    df["desc_digit"] = (df["desc_digit_count"] > 0).astype(int)

    df["desc_rm"] = df["item_description"].str.lower().str.contains(r"rm|[$]").astype(int)

    # -------------------------
    # 6) 존재 여부
    # -------------------------
    df["brand_missing"] = (df["brand_name"] == "unknown").astype(int)
    df["category_missing"] = (df["category_name"] == "unknown").astype(int)
    df["no_desc"] = df["item_description"].isin(["", "No description yet"]).astype(int)

    # -------------------------
    # 7) category_count
    # -------------------------
    df["category_count"] = df["category_name"].apply(lambda x: len(str(x).split("/")))

    # -------------------------
    # 8) 텍스트 lower
    # -------------------------
    df["name_lower"] = df["name"].str.lower()
    df["desc_lower"] = df["item_description"].str.lower()

    # -------------------------
    # 9) 이름/설명에 카테고리 포함 여부
    # -------------------------
    def has_category(row, col):
        cat = row["category_name"]
        if cat == "unknown":
            return 0
        text = row[col]
        for p in str(cat).split("/"):
            if p.lower().strip() in text:
                return 1
        return 0

    df["has_category_in_name"] = df.apply(lambda r: has_category(r, "name_lower"), axis=1)
    df["has_category_in_desc"] = df.apply(lambda r: has_category(r, "desc_lower"), axis=1)

    # -------------------------
    # 10) 이름/설명에 브랜드 포함 여부
    # -------------------------
    def has_brand(row, col):
        b = row["brand_name"].strip()
        if b == "" or b == "unknown":
            return 0
        return int(b in row[col])

    df["has_brand_in_name"] = df.apply(lambda r: has_brand(r, "name_lower"), axis=1)
    df["has_brand_in_desc"] = df.apply(lambda r: has_brand(r, "desc_lower"), axis=1)

    # -------------------------
    # 11) 라벨 인코딩
    # -------------------------
    for col in ["brand_filled", "cat1", "cat2", "cat3"]:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))

    return df


In [ ]:
mercari_df = build_features(mercari_df)


In [ ]:
valid_features = [c for c in features if c in mercari_df.columns]
print(valid_features)


In [ ]:
X = mercari_df[valid_features]
y = np.log1p(mercari_df["price"])


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np

params = {
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": 0.1,
    "num_leaves": 64,
    "max_depth": -1,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    "seed": 42,
    "verbose": -1,
}


In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

oof = np.zeros(len(X))
feature_importance = np.zeros(len(valid_features))
fold_rmse = []

for fold, (tr_idx, val_idx) in enumerate(kf.split(X), 1):
    print(f"===== Fold {fold} =====")

    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    dtrain = lgb.Dataset(X_tr, y_tr)
    dvalid = lgb.Dataset(X_val, y_val)

    model = lgb.train(
        params,
        dtrain,
        num_boost_round=5000,
        valid_sets=[dtrain, dvalid],
        valid_names=["train", "valid"],
        early_stopping_rounds=100,
        verbose_eval=200
    )

    pred_val = model.predict(X_val, num_iteration=model.best_iteration)
    oof[val_idx] = pred_val

    rmse = mean_squared_error(y_val, pred_val, squared=False)
    fold_rmse.append(rmse)
    print(f"Fold {fold} RMSE: {rmse:.5f}")

    feature_importance += model.feature_importance(importance_type="gain")


In [ ]:
print("\n===================================")
print("CV RMSE:", np.mean(fold_rmse))
print("Fold RMSEs:", fold_rmse)
print("===================================")


In [ ]:
fi_df = pd.DataFrame({
    "feature": valid_features,
    "importance": feature_importance / feature_importance.sum()
}).sort_values("importance", ascending=False)

fi_df.head(20)
